In [1]:
import pandas as pd

In [2]:

#train = pd.read_csv("train_stances.csv")
#train_body = pd.read_csv("train_bodies.csv")

In [3]:
train = pd.read_csv("competition_test_stances.csv")
group = train.groupby('Body ID')

In [4]:
train.head()

,Headline,Body ID,Stance
0,Ferguson riots: Pregnant woman loses eye after...,2008,unrelated
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,1550,unrelated
2,A Russian Guy Says His Justin Bieber Ringtone ...,2,unrelated
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",1793,unrelated
4,Argentina's President Adopts Boy to End Werewo...,37,unrelated


In [5]:
train_body = pd.read_csv("competition_test_bodies.csv")


In [6]:
group.get_group(1)

,Headline,Body ID,Stance
176,Apple installing safes in-store to protect gol...,1,unrelated
1647,El-Sisi denies claims he'll give Sinai land to...,1,agree
1932,Apple to keep gold Watch Editions in special i...,1,unrelated
3625,Apple Stores to Keep Gold “Edition” Apple Watc...,1,unrelated
3773,South Korean woman's hair 'eaten' by robot vac...,1,unrelated
5037,18-Karat Gold Apple Watch Edition To Be Locked...,1,unrelated
6755,"Apple Watch Price, Release Date News: Special ...",1,unrelated
9975,"Apple plans to phase out Beats Music brand, re...",1,unrelated
12418,Apple Stores will have custom-designed safes t...,1,unrelated
12823,"Polish Teenager Wakes Up During Brain Surgery,...",1,unrelated


In [7]:
train.sort_values('Body ID')[30:40]#['Headline']#[30:40]
train['Headline'][21704]

'Afghan woman kills 25 Taliban rebels to avenge her son’s murder'

In [8]:
train_body['articleBody']

0      Al-Sisi has denied Israeli reports stating tha...
1      A bereaved Afghan mother took revenge on the T...
2      CNBC is reporting Tesla has chosen Nevada as t...
3      A 4-inch version of the iPhone 6 is said to be...
4      GR editor’s Note\n\nThere are no reports in th...
                             ...                        
899    Congressional Republicans, evidently hoping th...
900    Did Obamacare work?\n\nIt’s worth reflecting u...
901    Millions may lose coverage next year if Congre...
902    Come November, the grim trudge across the incr...
903    Remember how much Republicans wanted to repeal...
Name: articleBody, Length: 904, dtype: object

In [9]:
train_body.head()

,Body ID,articleBody
0,1,Al-Sisi has denied Israeli reports stating tha...
1,2,A bereaved Afghan mother took revenge on the T...
2,3,CNBC is reporting Tesla has chosen Nevada as t...
3,12,A 4-inch version of the iPhone 6 is said to be...
4,19,GR editor’s Note\n\nThere are no reports in th...


In [10]:
from nltk.tokenize import TweetTokenizer
import re
tokenizer = TweetTokenizer()

In [13]:
import pandas as pd
import numpy as np
import copy
import re
#from keras.preprocessing.text import text_to_word_sequence
from nltk import WordNetLemmatizer


class BaseTokenizer(object):
    def process_text(self, text):
        raise NotImplemented

    def process(self, texts):
        for text in texts:
            yield self.process_text(text)


RE_PATTERNS = {
    ' american ':
        [
            'amerikan'
        ],

    ' adolf ':
        [
            'adolf'
        ],

    ' hitler ':
        [
            'hitler'
        ],

    ' fuck':
        [
            '(f)(u|[^a-z0-9 ])(c|[^a-z0-9 ])(k|[^a-z0-9 ])([^ ])*',
            '(f)([^a-z]*)(u)([^a-z]*)(c)([^a-z]*)(k)',
            ' f[!@#\$%\^\&\*]*u[!@#\$%\^&\*]*k', 'f u u c',
            '(f)(c|[^a-z ])(u|[^a-z ])(k)', r'f\*',
            'feck ', ' fux ', 'f\*\*',
            'f\-ing', 'f\.u\.', 'f###', ' fu ', 'f@ck', 'f u c k', 'f uck', 'f ck'
        ],

    ' ass ':
        [
            '[^a-z]ass ', '[^a-z]azz ', 'arrse', ' arse ', '@\$\$'
                                                           '[^a-z]anus', ' a\*s\*s', '[^a-z]ass[^a-z ]',
            'a[@#\$%\^&\*][@#\$%\^&\*]', '[^a-z]anal ', 'a s s'
        ],

    ' ass hole ':
        [
            ' a[s|z]*wipe', 'a[s|z]*[w]*h[o|0]+[l]*e', '@\$\$hole'
        ],

    ' bitch ':
        [
            'b[w]*i[t]*ch', 'b!tch',
            'bi\+ch', 'b!\+ch', '(b)([^a-z]*)(i)([^a-z]*)(t)([^a-z]*)(c)([^a-z]*)(h)',
            'biatch', 'bi\*\*h', 'bytch', 'b i t c h'
        ],

    ' bastard ':
        [
            'ba[s|z]+t[e|a]+rd'
        ],

    ' trans gender':
        [
            'transgender'
        ],

    ' gay ':
        [
            'gay'
        ],

    ' cock ':
        [
            '[^a-z]cock', 'c0ck', '[^a-z]cok ', 'c0k', '[^a-z]cok[^aeiou]', ' cawk',
            '(c)([^a-z ])(o)([^a-z ]*)(c)([^a-z ]*)(k)', 'c o c k'
        ],

    ' dick ':
        [
            ' dick[^aeiou]', 'deek', 'd i c k'
        ],

    ' suck ':
        [
            'sucker', '(s)([^a-z ]*)(u)([^a-z ]*)(c)([^a-z ]*)(k)', 'sucks', '5uck', 's u c k'
        ],

    ' cunt ':
        [
            'cunt', 'c u n t'
        ],

    ' bull shit ':
        [
            'bullsh\*t', 'bull\$hit'
        ],

    ' homo sex ual':
        [
            'homosexual'
        ],

    ' jerk ':
        [
            'jerk'
        ],

    ' idiot ':
        [
            'i[d]+io[t]+', '(i)([^a-z ]*)(d)([^a-z ]*)(i)([^a-z ]*)(o)([^a-z ]*)(t)', 'idiots'
                                                                                      'i d i o t'
        ],

    ' dumb ':
        [
            '(d)([^a-z ]*)(u)([^a-z ]*)(m)([^a-z ]*)(b)'
        ],

    ' shit ':
        [
            'shitty', '(s)([^a-z ]*)(h)([^a-z ]*)(i)([^a-z ]*)(t)', 'shite', '\$hit', 's h i t'
        ],

    ' shit hole ':
        [
            'shythole'
        ],

    ' retard ':
        [
            'returd', 'retad', 'retard', 'wiktard', 'wikitud'
        ],

    ' rape ':
        [
            ' raped'
        ],

    ' dumb ass':
        [
            'dumbass', 'dubass'
        ],

    ' ass head':
        [
            'butthead'
        ],

    ' sex ':
        [
            'sexy', 's3x', 'sexuality'
        ],

    ' nigger ':
        [
            'nigger', 'ni[g]+a', ' nigr ', 'negrito', 'niguh', 'n3gr', 'n i g g e r'
        ],

    ' shut the fuck up':
        [
            'stfu'
        ],

    ' pussy ':
        [
            'pussy[^c]', 'pusy', 'pussi[^l]', 'pusses'
        ],

    ' faggot ':
        [
            'faggot', ' fa[g]+[s]*[^a-z ]', 'fagot', 'f a g g o t', 'faggit',
            '(f)([^a-z ]*)(a)([^a-z ]*)([g]+)([^a-z ]*)(o)([^a-z ]*)(t)', 'fau[g]+ot', 'fae[g]+ot',
        ],

    ' mother fucker':
        [
            ' motha ', ' motha f', ' mother f', 'motherucker',
        ],

    ' whore ':
        [
            'wh\*\*\*', 'w h o r e'
        ],
}


class PatternTokenizer(BaseTokenizer):
    def __init__(self, lower=True,stem = True, initial_filters=r"[^a-z0-9!@#\$%\^\&\*_\-,\.']", patterns=RE_PATTERNS,
                 remove_repetitions=True):
        self.lower = lower
        self.stem = stem
        self.patterns = patterns
        self.initial_filters = initial_filters
        self.remove_repetitions = remove_repetitions

    def process_text(self, text):
        x = self._preprocess(text)
        for target, patterns in self.patterns.items():
            for pat in patterns:
                x = re.sub(pat, target, x)
        x = re.sub(r"[^a-z' ]", ' ', x)
        return x.split()

    def process_ds(self, ds):
        ### ds = Data series
        # lower
        ds = copy.deepcopy(ds)
        if self.lower:
            ds = ds.str.lower()
        
        # remove special chars
        if self.initial_filters is not None:
            ds = ds.str.replace(self.initial_filters, ' ')
        # fuuuuck => fuck
        if self.remove_repetitions:
            pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
            ds = ds.str.replace(pattern, r"\1")

        for target, patterns in self.patterns.items():
            for pat in patterns:
                ds = ds.str.replace(pat, target)

        ds = ds.str.replace(r"[^a-z' ]", ' ')
        corpus = []
        if self.stem:
            for words in ds.str.split():
                sentence = []
                for word in words:
                    sentence.append(WordNetLemmatizer().lemmatize(word,'v'))
                corpus.append(sentence)
        #return ds.str.split()
        #ds = pd.Series(corpus)
        ds = corpus
        corpus = []
        if self.stem:
            for words in ds:
                sentence = []
                for word in words:
                    sentence.append(WordNetLemmatizer().lemmatize(word,'n'))
                corpus.append(sentence)
        #return ds.str.split()
        ds = pd.Series(corpus)
        return ds
    def _preprocess(self, text):
        # lower
        if self.lower:
            text = text.lower()
        if self.stem:
            text = WordNetLemmatizer().lemmatize(text,'v')
        # remove special chars
        if self.initial_filters is not None:
            text = re.sub(self.initial_filters, ' ', text)

        # fuuuuck => fuck
        if self.remove_repetitions:
            pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
            text = pattern.sub(r"\1", text)
        return text


def main():
    #train = pd.read_json("train_gold.json",lines=True)
    #test = pd.read_json("test_unlabeled.json",lines=True)
    train = pd.read_csv("competition_test_stances.csv")
    train_body = pd.read_csv("competition_test_bodies.csv")

    tokenizer = PatternTokenizer()
    train["Headline"] = tokenizer.process_ds(train["Headline"]).str.join(sep=" ")
    train_body["articleBody"] = tokenizer.process_ds(train_body["articleBody"]).str.join(sep=" ")
    return train,train_body

In [14]:
train,body = main()

In [15]:
train.head()

,Headline,Body ID,Stance
0,ferguson riot pregnant woman lose eye after co...,2008,unrelated
1,crazy conservative be sure a gitmo detainee ki...,1550,unrelated
2,a russian guy say his justin bieber ringtone s...,2,unrelated
3,zombie cat bury kitty believe dead meow back t...,1793,unrelated
4,argentina's president adopt boy to end werewol...,37,unrelated


In [16]:
cat = ["agree","disagree","discuss","unrelated"]
#for i in range(len(train)):
#    for j in list_classes:
        

In [17]:
cat[0] == train['Stance'][4]

False

In [18]:
import numpy as np 
tt = np.zeros((len(train['Stance']),len(cat)))
for i in range(len(train['Stance'])):
    for j in range(len(cat)):
        if cat[j] == train['Stance'][i]:
            tt[i][j] = 1
        else:
            tt[i][j] = 0
df = {}
for j in range(len(tt[0])):        
    df[cat[j]] = tt[:,j]
df_ = pd.DataFrame(df)
train_ = pd.concat([train['Body ID'],train['Headline'], df_], axis=1)

In [19]:
train_.head()
#train_.to_csv('train_stance_pre.csv',index=False)
train_.to_csv('test_stance_pre.csv',index=False)
body.to_csv('test_body_pre.csv',index=False)

In [ ]:
classlabel = train.groupby('Stance')

In [ ]:
classlabel.size()

In [ ]:
body.head()

In [ ]:
body['articleBody'][body['Body ID']==0]

In [ ]:
for i in train_['Body ID']:
    

In [20]:
ttt = pd.read_csv('test_stance_pre.csv')
ttt.head()


,Body ID,Headline,agree,disagree,discuss,unrelated
0,2008,ferguson riot pregnant woman lose eye after co...,0.0,0.0,0.0,1.0
1,1550,crazy conservative be sure a gitmo detainee ki...,0.0,0.0,0.0,1.0
2,2,a russian guy say his justin bieber ringtone s...,0.0,0.0,0.0,1.0
3,1793,zombie cat bury kitty believe dead meow back t...,0.0,0.0,0.0,1.0
4,37,argentina's president adopt boy to end werewol...,0.0,0.0,0.0,1.0


In [ ]:
import nltk
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
words = ['gave','went','going','dating']
for word in words:
       print (word+"-->"+WordNetLemmatizer().lemmatize(word,'v'))

In [ ]:
WordNetLemmatizer().lemmatize('flying','v')